# Naive Bayes

En este notebook vamos a hacer un clasificador de correos _spam_ utilizando el algoritmo **Naive Bayes**. Para entrenar nuestro modelo vamos a usar los datos que encontramos [en el siguiente link](https://spamassassin.apache.org/old/publiccorpus/). Este es un _dataset_ de correos _spam_ y correos _ham_ (correo deseado).

Primero vamos a hacer una función que tome un mensaje y obtenga las palabras en él.

In [1]:
import re

def tokenize(text):
    text = text.lower()                         # Pasar a minúsculas
    all_words = re.findall("[a-z0-9']+", text)  # Extraer las palabras con expresiones regulares
    return set(all_words)                       # Con un Set no tenemos duplicados

tokenize("Hola! este es un mensaje de ejemplo :)")

{'de', 'ejemplo', 'es', 'este', 'hola', 'mensaje', 'un'}

Si ponemos atención, también nos elimina los caracteres especiales. Podemos hacer esto gracias al uso de expresiones regulares. Ahora necesitamos crear una clase que represente a los mensajes.

In [2]:
class Mail():
    def __init__(self, content, is_spam):
        self.content = content
        self.is_spam = is_spam

Y después de esto podemos definir nuestro clasificador de Bayes. Sin embargo, vamos a hacer una modificación respecto a lo que vimos en clases. En este caso vamos a predecir el evento $S$: el mensaje es _spam_. Vamos a suponer que la probabilidad de que un mensaje sea _spam_ o no lo sea es la misma, 0.5. Luego formaremos un conjunto gigante con todo el universo de palabras, y así, tendremos el evento $X_i$ que significa si la palabra $i$ de nuestro vocabulario aparece en un correo.

Así, si tenemos $n$ palabras, lo que queremos calcular es $P(S | X_1, \dots, X_n)$. Entonces para determinar si un correo es _spam_, tendremos que multiplicar las probabilidades $P(X_i | S)$ y $P(X_i | ¬S)$. Como se ve, debemos pararnos en cada palabra de nuestro universo, viendo si esta está o no en el correo, y buscando la probabilidad asociada.

Aquí vienen las modificaciones:

- Lo primero es que en general **no nos gusta** multiplicar números pequeños. Así que transformaremos las multiplicaciones en sumas gracias a la función logaritmo.
- Lo segundo es que podríamos pensar que P(X_i | S) lo podemos calcular como la fracción de mensajes _spam_ que contienen a la palabra $i$. Pero si tenemos una palabra que no aparece en ningún mensaje _spam_, vamos a hacer que para cualquier correo con esa palabra asignemos la probabilidad 0 de que sea spam (al multiplicar hacemos todo 0). Por esto, hacemos el siguiente truco:

$$
P(X_i | S) = \frac{(k + \text{número de spams que contienen la palabra}\ i)}{2k + \text{número de spams}}
$$

Donde $k$ es un número que vamos a escoger a mano.

In [6]:
from collections import defaultdict # Diccionario con valores default
import math

class NaiveBayesClassifier:
    def __init__(self, k=0.5):
        self.k = k  # smoothing factor

        self.tokens = set()
        self.token_spam_counts = defaultdict(int)
        self.token_ham_counts = defaultdict(int)
        self.spam_messages = 0
        self.ham_messages = 0

    # Función para entrenar
    def train(self, messages):
        for message in messages:
            # Increment message counts
            if message.is_spam:
                self.spam_messages += 1
            else:
                self.ham_messages += 1

            # Increment word counts
            for token in tokenize(message.content):
                self.tokens.add(token)
                if message.is_spam:
                    self.token_spam_counts[token] += 1
                else:
                    self.token_ham_counts[token] += 1

    # Función que calcula las probabilidades asociadas a cada palabra
    # Si la palabra no está usaremos (1 - la probabilidad)
    def probabilities(self, token):
        # Retorna P(token | spam) y P(token | not spam)
        spam = self.token_spam_counts[token]
        ham = self.token_ham_counts[token]

        p_token_spam = (spam + self.k) / (self.spam_messages + 2 * self.k)
        p_token_ham = (ham + self.k) / (self.ham_messages + 2 * self.k)

        return p_token_spam, p_token_ham

    # Función para predecir un texto
    def predict(self, text):
        text_tokens = tokenize(text)
        log_prob_if_spam = 0.0
        log_prob_if_ham = 0.0

        # Vemos todo nuestro vocabulario
        for token in self.tokens:
            prob_if_spam, prob_if_ham = self.probabilities(token)

            # Si el token aparece en el mensaje sumamos el log de la probabilidad asociada
            if token in text_tokens:
                log_prob_if_spam += math.log(prob_if_spam)
                log_prob_if_ham += math.log(prob_if_ham)

            # En otro caso, sumamos el log de la probabilidad de no haberlo visto: log(1 - probabilidad de ver el mensaje)
            else:
                log_prob_if_spam += math.log(1.0 - prob_if_spam)
                log_prob_if_ham += math.log(1.0 - prob_if_ham)

        prob_if_spam = math.exp(log_prob_if_spam)
        prob_if_ham = math.exp(log_prob_if_ham)
        return prob_if_spam / (prob_if_spam + prob_if_ham)
    

En la clase anterior, `predict` nos va a arrojar una probabilidad. Un correo será _spam_ si la probabilidad es mayor o igual a 0.5. Es buena idea que te detengas a pensar por qué esto es así. Ahora vamos a probar nuestros modelos con unos mensajes de prueba.

In [25]:
messages = [
    Mail("compra bitcoin", is_spam=True),
    Mail("bitcoin a la baja", is_spam=True),
    Mail("compra pan", is_spam=False),
    Mail("no soy spam", is_spam=False)
]

model = NaiveBayesClassifier(k=0.5)
model.train(messages)

model.predict("has escuchado de bitcoin?")

0.8928571428571429

## Cargando el dataset

Ahora vamos a considerar los correos del _link_ que presentamos más arriba. Estos están en la carpeta _mails_ junto a este notebook. Vamos a presentarte el código que te ayudará a leerlos. En este ejemplo vamos a trabajar solo con el _Subject_ del correo.

In [24]:
import random

# Función para hacer split de un dataset
def split_data(data, perc):
    data = data[:]                    # Copiamos el dataset
    random.shuffle(data)              # porque shuffle modifica la lista, aquí la ordenamos de forma aleatoria
    cut = int(len(data) * perc)       # Vemos hasta que posición tomamos
    return data[:cut], data[cut:]     # Retornamos las dos partes

PATH_MAILS = 'mails/*/*'

data = []

import glob, re

# Recorremos todos los archivos
for filename in glob.glob(PATH_MAILS):
    # Checkeamos si es spam o no
    is_spam = 'ham' not in filename
    
    # Ignoramos los posibles errores al abrir un archivo
    with open(filename, errors='ignore') as mail_file:
        for line in mail_file:
            if line.startswith("Subject:"):
                # Hacemos strip desde la izquierda
                subject = line.lstrip("Subject: ")
                data.append(Mail(subject, is_spam))
                # Después de añadir el subject terminamos
                break
                
# Generamos dos listas de mensajes
# Una de entrenamiento y otra de prueba
train_mails, test_mails = split_data(data, 0.7)